In [ ]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import gdal

In [ ]:
    def load_clean_yield_data(yield_data_filepath):
        """
        Cleans the yield data by making sure any Nan values in the columns we care about
        are removed
        """
        important_columns=['Year', 'State ANSI', 'County ANSI', 'Value']
        yield_data=pd.read_csv(yield_data_filepath).dropna(subset=important_columns,how='any')

        return yield_data

    def get_tif_files(image_path):
        """
        Get all the .tif files in the image folder.

        Parameters
        ----------
        image_path: pathlib Path
            Directory to search for tif files
        Returns:
            A list of .tif filenames
        """
        files = []
        for dir_file in image_path.iterdir():
            if str(dir_file).endswith('tif'):
                files.append(str(dir_file.parts[-1]))
        return files
    def get_tif_files_12(image_path):#name length less than 12
        files = []
        for dir_file in image_path.iterdir():
            if str(dir_file).endswith('tif'):
                if len(str(dir_file.parts[-1]))<12:#max possible name is ss_ccc.tif, len=10
                    files.append(str(dir_file.parts[-1]))
        return files
    def get_tif_files_12(mask_path,temperature_path,image_path,weather_path):
        mask_files=[f for f in listdir(mask_path) if isfile(join(mask_path, f))]
        temperature_files=[f for f in listdir(temperature_path) if isfile(join(temperature_path, f))]
        image_files=[f for f in listdir(image_path) if isfile(join(image_path, f))]
        weather_files=[f for f in listdir(weather_path) if isfile(join(weather_path, f))]
        files = []
        for dir_file in image_path.iterdir():
            if str(dir_file).endswith('tif'):
                if len(str(dir_file.parts[-1]))<12:#max possible name is ss_ccc.tif, len=10
                    if str(dir_file.parts[-1]) in temperature_files:
                        if str(dir_file.parts[-1]) in weather_files:
                            if str(dir_file.parts[-1]) in mask_files:
                                files.append(str(dir_file.parts[-1]))
        return files


In [ ]:
#feature engineering goes here
from pathlib import Path
import numpy as np
import pandas as pd
import math

class Engineer:
    """
    Take the preprocessed data from the Data Cleaner
    and turn the images into matrices which can be input
    into the machine learning models.

    These matrices can either be histograms, which describe the distributions of
    pixels on each band, and contain 32 bins.
    This turns the band of an image from dim=(width*height) to dim=32.

    They can also be means of each band, which turns the band of an image from
    dim=(width*height) into a scalar value.
    """
    def __init__(self, cleaned_data_path=Path('img_output'),
                 yield_data_filepath=Path('SOP- Time Series Analysis with Deep Learning/yield_data_less_entries.csv'),
                 county_data_filepath=Path('SOP- Time Series Analysis with Deep Learning/county_data.csv')):
        self.cleaned_data=cleaned_data_path
        self.processed_files=global_processed_files
        print(len(self.processed_files))
        #merge the yield and county data for easier manipulation
        yield_data=load_clean_yield_data(yield_data_filepath)[['Year','State ANSI','County ANSI','Value']]
        yield_data.columns=['Year','State','County','Value']
        county_data=pd.read_csv(county_data_filepath)[['CntyFips','StateFips', 'Longitude','Latitude']]
        county_data.columns=['County', 'State', 'Longitude', 'Latitude']
        self.yield_data=yield_data.merge(county_data, how='left', on=['County','State'])

    def get_filenames(self):
        """
        Get all the .tif files in the image folder.
        """
        files=[]
        
        print(self.cleaned_data)
        for dir_file in Path(self.cleaned_data).iterdir():
            if str(dir_file).endswith('npy'):
                #strip out the directory so it's just file name
                files.append(str(dir_file.parts[-1]))
                global_processed_files.append(str(dir_file.parts[-1]))
        # files.sort()#to ensure that soil and image data stored in same sequence
        # global_processed_files=files
        return files
    
    @staticmethod
    def filter_timespan(imcol, start_day=91, end_day=335, composite_period=8, bands=11):
        # 1st April to 30th Nov
        """
        Given an image collection containing a year's worth of data,
        filter it between start_day and end_day. If end_day is later than the date
        for which we have data, the image collection is padded with zeros.

        Parameters
        ----------
        imcol: The image collection to be filtered
        start_day: int, default=49
            The earliest day for which to consider data
        end_day: int, default=305
            The last day for which to consider data
        composite_period: int, default=8
            The composite period of the images. Default taken from the composite
            periods of the MOD09A1 and MYD11A2 datasets
        bands: int, default=9
            The number of bands per image. Default taken from the number of bands in the
            MOD09A1 + the number of bands in the MYD11A2 datasets

        Returns
        ----------
        A filtered image collection
        """
        start_index=int(math.floor(start_day/composite_period))*bands
        end_index=int(math.floor(end_day/composite_period))*bands

        if end_index>imcol.shape[2]:
            padding=np.zeros((imcol.shape[0], imcol.shape[1], end_index-imcol.shape[2]))
            imcol=np.concatenate((imcol,padding), axis=2)
        return imcol[:,:, start_index:end_index]

    @staticmethod
    def _calculate_histogram(imagecol, num_bins=32, bands=11, max_bin_val=4999,
                             channels_first=True):
        """
        Given an image collection, turn it into a histogram.

        Parameters
        ----------
        imcol: The image collection to be histogrammed
        num_bins: int, default=32
            The number of bins to use in the histogram.
        bands: int, default=9
            The number of bands per image. Default taken from the number of bands in the
            MOD09A1 + the number of bands in the MYD11A2 datasets
        max_bin_val: int, default=4999
            The maximum value of the bins. The default is taken from the original repository;
            note that the maximum pixel values from the MODIS datsets range from 16000 to
            18000 depending on the band

        Returns
        ----------
        A histogram for each band, of the band's pixel values. The output shape is
        [num_bins, times, bands], where times is the number of unique timestamps in the
        image collection.
        """
        bin_seq=np.linspace(1,max_bin_val, num_bins+1)

        hist=[]
        for im in np.split(imagecol, imagecol.shape[-1]/bands, axis=-1):
            imhist=[]
            for i in range(im.shape[-1]):
                density,_=np.histogram(im[:,:,i],bin_seq, density=False)
                # max prevents divide by zero
                imhist.append(density/max(1,density.sum()))
            if channels_first:
                hist.append(np.stack(imhist))
            else:
                hist.append(np.stack(imhist,axis=1))
        return np.stack(hist, axis=1)

    def process(self, num_bands=11, generate='histogram', num_bins=32, max_bin_val=4999,
                channels_first=True):
        """
        Parameters
        ----------
        num_bands: int, default=9
            The number of bands per image. Default taken from the number of bands in the
            MOD09A1 + the number of bands in the MYD11A2 datasets
        generate: str, {'mean', 'histogram'}, default='mean'
            What to generate from the data. If 'mean', calculates a mean
            of all the bands. If 'histogram', calculates a histogram of all
            the bands with num_bins bins for each band.
        num_bins: int, default=32
            If generate=='histogram', the number of bins to generate in the histogram.
        max_bin_val: int, default=4999
            The maximum value of the bins. The default is taken from the original repository;
            note that the maximum pixel values from the MODIS datsets range from 16000 to
            18000 depending on the band
        channels_first: boolean, default=True
            If true, the output histogram has shape [bands, times, bins]. Otherwise, it
            has shape [times, bins, bands]
        """
        #define all the outputs of this method
        output_images=[]
        yields=[]
        years=[]
        locations=[]
        state_county_info=[]
        count=0
        for yield_data in self.yield_data.itertuples():
            year=yield_data.Year
            county=yield_data.County
            state=yield_data.State

            filename=f'{year}_{int(state)}_{int(county)}.npy'
            if filename in self.processed_files:
                image=np.load(self.cleaned_data/filename)
                image=self.filter_timespan(image, start_day=91, end_day=335,
                                           bands=num_bands)
                # from 1st April to 30th Nov
                print(filename,"\t",image.shape)
                if generate=='mean':
                    image=np.sum(image, axis=(0,1))/np.count_nonzero(image)*image.shape[2]
                    image[np.isnan(image)]=0
                elif generate=='histogram':
                    image=self._calculate_histogram(image, bands=num_bands,
                                                    num_bins=num_bins,
                                                    max_bin_val=max_bin_val,
                                                    channels_first=channels_first)
                    
                output_images.append(image)
                yields.append(yield_data.Value)
                years.append(year)

                # some of the minus signs in the longitudes have been carried over to the
                # longitudes, i.e. 19.683885, -155.393159 becomes 19.683885-, 155.393159
                try:
                    lat, lon=float(yield_data.Latitude), float(yield_data.Longitude)
                except ValueError:
                    lat=float(yield_data.Latitude[:-1])
                    lon=-float(yield_data.Longitude)
                locations.append(np.array([lon,lat]))
                state_county_info.append(np.array([int(state), int(county)]))
                
        np.savez(self.cleaned_data/f'histogram_all_{"mean" if(generate=="mean") else "full"}.npz',
                 output_image=np.stack(output_images), output_yield=np.array(yields),
                 output_year=np.array(years), output_locations=np.stack(locations),
                 output_index=np.stack(state_county_info))
        print(f'Finished generating image {generate}s!')


In [ ]:

cleaned_data_path=Path('/content/drive/MyDrive/cleanedData/imageComposite')
yield_data_path = Path('/content/drive/MyDrive/data_Lakshya/corn_yield_Lakshya.csv')
county_data_path = Path('/content/drive/MyDrive/data_Lakshya/county_data.csv')
num_bins=32
max_bin_val=5000

engineer = Engineer(cleaned_data_path, yield_data_path, county_data_path)
engineer.process(num_bands=11, generate='histogram', num_bins=num_bins, max_bin_val=max_bin_val,
                channels_first=True)

In [4]:
import numpy as np
bin_seq=np.linspace(1,5000, 32+1)
print(bin_seq)

[1.00000000e+00 1.57218750e+02 3.13437500e+02 4.69656250e+02
 6.25875000e+02 7.82093750e+02 9.38312500e+02 1.09453125e+03
 1.25075000e+03 1.40696875e+03 1.56318750e+03 1.71940625e+03
 1.87562500e+03 2.03184375e+03 2.18806250e+03 2.34428125e+03
 2.50050000e+03 2.65671875e+03 2.81293750e+03 2.96915625e+03
 3.12537500e+03 3.28159375e+03 3.43781250e+03 3.59403125e+03
 3.75025000e+03 3.90646875e+03 4.06268750e+03 4.21890625e+03
 4.37512500e+03 4.53134375e+03 4.68756250e+03 4.84378125e+03
 5.00000000e+03]


In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/cleanedData/histImageComposite/histogram_all_full.npz')
lst = data.files
np.set_printoptions(threshold=np. inf)

#Required output in the form (1(years), 32(bins), 11(layers), days) and currently shape is (years, layers, days, bins(32))
#required (10,32,11,19)

for item in lst:
    print(item)
    a = data[item][0]
    print(data[item][0].shape)
    print(data[item][0])
print(len(data["output_index"]))

In [ ]:
out_img_composite = data['output_image']
out_img_composite.shape

In [ ]:
#data preparation IMPORTANT TRANSPOSE
out_img_composite = np.transpose(out_img_composite, axes=(0,2,3,1))
out_img_composite.shape

In [ ]:
out_img_composite = out_img_composite.reshape(out_img_composite.shape[0],out_img_composite.shape[1],1,out_img_composite.shape[2],out_img_composite.shape[3])
out_img_composite.shape

### Engineering for Soil

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import math

class EngineerSoil:
    """
    Take the preprocessed data from the Data Cleaner
    and turn the images into matrices which can be input
    into the machine learning models.

    These matrices can either be histograms, which describe the distributions of
    pixels on each band, and contain 32 bins.
    This turns the band of an image from dim=(width*height) to dim=32.

    They can also be means of each band, which turns the band of an image from
    dim=(width*height) into a scalar value.
    """
    def __init__(self, cleaned_data_path=Path('img_output'),
                 yield_data_filepath=Path('SOP- Time Series Analysis with Deep Learning/yield_data_less_entries.csv'),
                 county_data_filepath=Path('SOP- Time Series Analysis with Deep Learning/county_data.csv')):
        self.cleaned_data=cleaned_data_path
        self.processed_files=global_processed_files
        print(len(self.processed_files))
        print(self.processed_files)
        #merge the yield and county data for easier manipulation
        yield_data=load_clean_yield_data(yield_data_filepath)[['Year','State ANSI','County ANSI','Value']]
        yield_data.columns=['Year','State','County','Value']
        county_data=pd.read_csv(county_data_filepath)[['CntyFips','StateFips', 'Longitude','Latitude']]
        county_data.columns=['County', 'State', 'Longitude', 'Latitude']
        self.yield_data=yield_data.merge(county_data, how='left', on=['County','State'])

    def get_filenames(self):
        """
        Get all the .tif files in the image folder.
        """
        files=[]
        files.sort()#to ensure that soil and image data stored in same sequence
        for dir_file in Path(self.cleaned_data).iterdir():
            if str(dir_file).endswith('npy'):
                if str(dir_file.parts[-1]) in global_processed_files:
                    files.append(str(dir_file.parts[-1]))
        return files
    
    @staticmethod
    def filter_timespan(imcol, start_day=49, end_day=305, composite_period=8, bands=9):
        """
        Given an image collection containing a year's worth of data,
        filter it between start_day and end_day. If end_day is later than the date
        for which we have data, the image collection is padded with zeros.

        Parameters
        ----------
        imcol: The image collection to be filtered
        start_day: int, default=49
            The earliest day for which to consider data
        end_day: int, default=305
            The last day for which to consider data
        composite_period: int, default=8
            The composite period of the images. Default taken from the composite
            periods of the MOD09A1 and MYD11A2 datasets
        bands: int, default=9
            The number of bands per image. Default taken from the number of bands in the
            MOD09A1 + the number of bands in the MYD11A2 datasets

        Returns
        ----------
        A filtered image collection
        """
        start_index=int(math.floor(start_day/composite_period))*bands
        end_index=int(math.floor(end_day/composite_period))*bands

        if end_index>imcol.shape[2]:
            padding=np.zeros((imcol.shape[0], imcol.shape[1], end_index-imcol.shape[2]))
            imcol=np.concatenate((imcol,padding), axis=2)
        return imcol[:,:, start_index:end_index]

    @staticmethod
    def _calculate_histogram(imagecol, num_bins=32, bands=11, max_bin_val=4999,
                             channels_first=True):
        """
        Given an image collection, turn it into a histogram.

        Parameters
        ----------
        imcol: The image collection to be histogrammed
        num_bins: int, default=32
            The number of bins to use in the histogram.
        bands: int, default=9
            The number of bands per image. Default taken from the number of bands in the
            MOD09A1 + the number of bands in the MYD11A2 datasets
        max_bin_val: int, default=4999
            The maximum value of the bins. The default is taken from the original repository;
            note that the maximum pixel values from the MODIS datsets range from 16000 to
            18000 depending on the band

        Returns
        ----------
        A histogram for each band, of the band's pixel values. The output shape is
        [num_bins, times, bands], where times is the number of unique timestamps in the
        image collection.
        """
        bin_seq=np.linspace(1,max_bin_val, num_bins+1)

        hist=[]
        for im in np.split(imagecol, imagecol.shape[-1]/bands, axis=-1):
            imhist=[]
            for i in range(im.shape[-1]):
                density,_=np.histogram(im[:,:,i],bin_seq, density=False)
                # max prevents divide by zero
                imhist.append(density/max(1,density.sum()))
            if channels_first:
                hist.append(np.stack(imhist))
            else:
                hist.append(np.stack(imhist,axis=1))
        return np.stack(hist, axis=1)

    def process(self, num_bands=36, generate='histogram', num_bins=32, max_bin_val=4999,
                channels_first=True):
        """
        Parameters
        ----------
        num_bands: int, default=9
            The number of bands per image. Default taken from the number of bands in the
            MOD09A1 + the number of bands in the MYD11A2 datasets
        generate: str, {'mean', 'histogram'}, default='mean'
            What to generate from the data. If 'mean', calculates a mean
            of all the bands. If 'histogram', calculates a histogram of all
            the bands with num_bins bins for each band.
        num_bins: int, default=32
            If generate=='histogram', the number of bins to generate in the histogram.
        max_bin_val: int, default=4999
            The maximum value of the bins. The default is taken from the original repository;
            note that the maximum pixel values from the MODIS datsets range from 16000 to
            18000 depending on the band
        channels_first: boolean, default=True
            If true, the output histogram has shape [bands, times, bins]. Otherwise, it
            has shape [times, bins, bands]
        """

        #define all the outputs of this method
        output_images=[]
        yields=[]
        years=[]
        locations=[]
        state_county_info=[]
        count=0
        for yield_data in self.yield_data.itertuples():
            year=yield_data.Year
            county=yield_data.County
            state=yield_data.State

            filename=f'{year}_{int(state)}_{int(county)}.npy'
            if filename in self.processed_files:
                image=np.load(self.cleaned_data/filename)
                print(filename,"\t",image.shape)
                if generate=='mean':
                    image=np.sum(image, axis=(0,1))/np.count_nonzero(image)*image.shape[2]
                    image[np.isnan(image)]=0
                elif generate=='histogram':
                    image=self._calculate_histogram(image, bands=num_bands,
                                                    num_bins=num_bins,
                                                    max_bin_val=max_bin_val,
                                                    channels_first=channels_first)
                    
                output_images.append(image)
                yields.append(yield_data.Value)
                years.append(year)

                # some of the minus signs in the longitudes have been carried over to the
                # longitudes, i.e. 19.683885, -155.393159 becomes 19.683885-, 155.393159
                try:
                    lat, lon=float(yield_data.Latitude), float(yield_data.Longitude)
                except ValueError:
                    lat=float(yield_data.Latitude[:-1])
                    lon=-float(yield_data.Longitude)
                locations.append(np.array([lon,lat]))

                state_county_info.append(np.array([int(state), int(county)]))

        np.savez(self.cleaned_data/f'histogram_all_{"mean" if(generate=="mean") else "full"}.npz',
                 output_image=np.stack(output_images), output_yield=np.array(yields),
                 output_year=np.array(years), output_locations=np.stack(locations),
                 output_index=np.stack(state_county_info))
        print(f'Finished generating image {generate}s!')


In [ ]:
# cleaned_data_path = Path('/content/drive/MyDrive/singleData')
cleaned_data_path=Path('/content/drive/MyDrive/cleanedData/histSoilComposite')
yield_data_path = Path('/content/drive/MyDrive/data_Lakshya/corn_yield_Lakshya.csv')
county_data_path = Path('/content/drive/MyDrive/data_Lakshya/county_data.csv')
num_bins=32
max_bin_val=4999

engineerSoil = EngineerSoil(cleaned_data_path, yield_data_path, county_data_path)
engineerSoil.process(num_bands=36, generate='histogram', num_bins=num_bins, max_bin_val=max_bin_val,
                channels_first=True)

In [ ]:
data = np.load('/content/drive/MyDrive/cleanedData/soilComposite/histogram_all_full.npz')
lst = data.files
np.set_printoptions(threshold=np. inf)

#Required output in the form (1(years), 32(bins), 11(layers), days) and currently shape is (years, layers, days, bins(32))
#required (10,32,11,19)

for item in lst:
    print(item)
    a = data[item][0]
    print(data[item][0].shape)
print(len(data["output_index"]))

In [ ]:
out_soil = data['output_image']
out_soil.shape

In [ ]:
out_soil = np.transpose(out_soil, axes=(0,2,3,1))
out_soil.shape